In [1]:
# This notebook checks MNE functions for ECG and EOG detection
# Cropped data is used here (5 minutes only), tried on whole data - takes forever.


#Load data, make folders
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import mne
from mne.time_frequency import tfr_morlet, psd_multitaper, psd_welch
import configparser
import ancpbids

from main_meg_qc import save_derivative_html, sanity_check, initial_stuff
from RMSE_meq_qc import MEG_QC_rmse
from PSD_meg_qc import PSD_QC 


In [ ]:
#%% TRY SEPARATE FUNCTIONS HERE

config = configparser.ConfigParser()
config.read('settings.ini')
#data_file='/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/data/sub_HT05ND16/210811/mikado-1.fif'
data_file='/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/data/ds004276/sub-001/meg/sub-001_task-words_meg.fif'
sid='001'

df_epochs, epochs_mg, channels, raw_bandpass, raw_bandpass_resamp, raw_cropped, raw = initial_stuff(config, data_file)
m_or_g_chosen = ['mags', 'grads']

m_or_g_chosen = sanity_check(m_or_g_chosen, channels)

if len(m_or_g_chosen) == 0: 
    raise ValueError('No channels to analyze. Check presence of mags and grads in your data set and parameter do_for in settings.')

_, list_of_figures, list_of_fig_desc = MEG_QC_rmse(sid, config, channels, df_epochs, raw_bandpass_resamp, m_or_g_chosen)

In [2]:
#%% TRY TO SAVE DERIVATIVE WITH BIDS DATASET HERE:
config = configparser.ConfigParser()
config.read('settings.ini')

direct = config['DEFAULT']['data_directory']
dataset_path = ancpbids.utils.fetch_dataset(direct)

from ancpbids import BIDSLayout
layout = BIDSLayout(dataset_path)

# list_of_fifs = layout.get(suffix='meg', extension='.fif', return_type='filename')

list_of_subs = layout.get_subjects()

# list_of_entities = layout.get_entities()
# print(list_of_entities)

save_derivative_html(dataset_path, list_of_subs)

Opening raw data file /Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/data/ds004276/sub-001/meg/sub-001_task-words_meg.fif...
    Range : 0 ... 467999 =      0.000 ...   467.999 secs
Ready.
No events with set minimum duration were found using all stimulus channels. No epoching can be done. Try different event duration in config file.
There are no gradiometers in this data set: check parameter do_for in config file. Analysis will be done only for magnetometers.
RMSE per epoch can not be calculated because no events are present. Check stimulus channel.
Opening raw data file /Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/data/ds004276/sub-002/meg/sub-002_task-words_meg.fif...
    Range : 0 ... 1101999 =      0.000 ...  1101.999 secs
Ready.
No events with set minimum duration were found using all stimulus channels. No epoching can be done. Try different event duration in config file.
RMSE per epoch can not be calculated because no events are present. Check stimulus cha

In [ ]:
#%%
# Try on one subject:
subj = '001'
data_file0 = layout.get(suffix='meg', extension='.fif', return_type='filename', subj=subj)[0]
print('Try now ', data_file0)

n_events, df_epochs_mags, df_epochs_grads, epochs_channels_mags, epochs_channels_grads, channels, filtered_d, filtered_d_resamp, raw_cropped, raw = initial_stuff(config, data_file=data_file0)

print('NOW START THE QC stuff)')

#list_of_figures_PSD, list_of_fig_descriptions_PSD = MEG_QC_measures(subj, config, n_events, df_epochs_mags, df_epochs_grads, epochs_channels_mags, epochs_channels_grads, channels, filtered_d, filtered_d_resamp, raw_cropped, raw)

#%%

from PSD_meg_qc import Freq_Spectrum_meg

print(len(channels['mags']))
freqs, psds, fig_path_psd, fig_psd, fig_desc = Freq_Spectrum_meg(data=filtered_d_resamp, m_or_g = 'mags', sid=subj, freq_min=0.5, freq_max=100, n_fft=1000, n_per_seg=1000, freq_tmin=None, freq_tmax=None, ch_names=channels['mags'])